In [ ]:
!pip install -q sentence_transformers

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pickle

In [ ]:
import numpy as np

def softmax(x):
    """
    Compute softmax values for each set of scores in x.

    Parameters:
    x (array-like): Input array of scores.

    Returns:
    np.ndarray: Softmax probabilities.
    """
    exp_x = np.exp(x - np.max(x))  # Subtract max for numerical stability
    return exp_x / np.sum(exp_x)


In [ ]:
full_data_for_ranking = pd.read_excel('/content/drive/MyDrive/datasets/Budget_Speech_Sector_Company/sectorwise_budgetdaywise_performance_with_text_ranked_full.xlsx')
full_data_for_ranking

date          year budget_full_text_filename                 sector  \
0   1997-02-28     1997-1998              bs199798.txt                Finance   
1   1997-02-28     1997-1998              bs199798.txt               Textiles   
2   1998-01-06     1998-1999              bs199899.txt              Chemicals   
3   1998-01-06     1998-1999              bs199899.txt       Auto Ancillaries   
4   1998-03-25  1998-1999(I)           bs199899(I).txt       Auto Ancillaries   
..         ...           ...                       ...                    ...   
424 2025-01-02     2025-2026             bs2025_26.txt            Fertilizers   
425 2025-01-02     2025-2026             bs2025_26.txt  Air Transport Service   
426 2025-01-02     2025-2026             bs2025_26.txt         Agro Chemicals   
427 2025-01-02     2025-2026             bs2025_26.txt                Leather   
428 2025-01-02     2025-2026             bs2025_26.txt          IT - Hardware   

                                          text_segment  number_of_companies  \
0    ['Proposal to reduce corporate tax rates: Dome...                    2   
1    ['Reduction of duty on synthetic rubber, fibre...                    1   
2    ['The duty on photographic chemicals is being ...                    1   
3    ["Industrial deregulation would remain incompl...                    1   
4    ['industry continues to be in the doldrums ave...                    1   
..                                                 ...                  ...   
424  ['National Mission on High Yielding Seeds will...                   19   
425  ['Modified UDAN scheme to enhance regional con...                    2   
426  ['National Mission on High Yielding Seeds will...                   21   
427  ['Focus Product Scheme for Footwear & Leather ...                   12   
428  ['Reduction in BCD on Carrier-Grade Ethernet S...                    7   

     majority_target_binary  mean_target_binary  \
0                       1.0            1.000000   
1                       1.0            1.000000   
2                       1.0            1.000000   
3                       0.0            0.000000   
4                       0.0            0.000000   
..                      ...                 ...   
424                     1.0            0.526316   
425                     0.5            0.500000   
426                     0.0            0.476190   
427                     0.0            0.416667   
428                     0.0            0.285714   

     mean_target_percentage_increment  rank  
0                            0.139282     1  
1                            0.003745     2  
2                            0.026786     1  
3                           -0.101090     2  
4                           -0.033730     1  
..                                ...   ...  
424                          0.001219    21  
425                          0.005490    22  
426                          0.000646    23  
427                         -0.002770    24  
428                         -0.007205    25  

[429 rows x 10 columns]

In [ ]:
yr1 = 2020
yr2 = 2024

full_data_for_ranking['year_use'] = full_data_for_ranking['year'].apply(lambda x : int(str(x)[:4]))
full_data_for_ranking['text_segment'] = full_data_for_ranking['text_segment'].apply(lambda x : eval(x))
full_data_for_ranking['text_use'] = full_data_for_ranking.apply(lambda x : x.sector + ' [SEP] ' +' '.join(x.text_segment), axis = 1)

train = full_data_for_ranking[full_data_for_ranking['year_use']<yr1]
valid = full_data_for_ranking[(full_data_for_ranking['year_use']>=yr1) & (full_data_for_ranking['year_use']<yr2)]
test = full_data_for_ranking[full_data_for_ranking['year_use']>=yr2]
train.shape, valid.shape, test.shape

((318, 12), (41, 12), (70, 12))

In [ ]:
train['label'] = train['mean_target_percentage_increment'].apply(lambda x : 0 if x < 0 else 1)
valid['label'] = valid['mean_target_percentage_increment'].apply(lambda x : 0 if x < 0 else 1)
test['label'] = test['mean_target_percentage_increment'].apply(lambda x : 0 if x < 0 else 1)

In [ ]:
from sentence_transformers import SentenceTransformer
import torch.nn.functional as F


matryoshka_dim = 128
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)




In [ ]:
train_texts = ['classification: ' + i for i in train['text_use'].tolist()]

embeddings = model.encode(train_texts,convert_to_tensor=True)
embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
embeddings = embeddings[:, :matryoshka_dim]
train_embeddings = F.normalize(embeddings, p=2, dim=1)

In [ ]:
valid_texts = ['classification: ' + i for i in valid['text_use'].tolist()]

embeddings = model.encode(valid_texts, convert_to_tensor=True)
embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
embeddings = embeddings[:, :matryoshka_dim]
valid_embeddings = F.normalize(embeddings, p=2, dim=1)

test_texts =  ['classification: ' + i for i in test['text_use'].tolist()]

embeddings = model.encode(test_texts, convert_to_tensor=True)
embeddings = F.layer_norm(embeddings, normalized_shape=(embeddings.shape[1],))
embeddings = embeddings[:, :matryoshka_dim]
test_embeddings = F.normalize(embeddings, p=2, dim=1)

In [ ]:
pickle.dump(train_embeddings.cpu().detach().numpy(), open('/content/drive/MyDrive/datasets/Budget_Speech_Sector_Company/train_embeddings.pkl','wb'))
pickle.dump(valid_embeddings.cpu().detach().numpy(), open('/content/drive/MyDrive/datasets/Budget_Speech_Sector_Company/valid_embeddings.pkl','wb'))
pickle.dump(test_embeddings.cpu().detach().numpy(), open('/content/drive/MyDrive/datasets/Budget_Speech_Sector_Company/test_embeddings.pkl','wb'))

In [ ]:
# Example usage
scores = np.array(test['mean_target_percentage_increment'])
test_scores_softmax= softmax(scores)

# Train Logistic Regression Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(train_embeddings.cpu().detach().numpy(), train['label'].tolist())
valid_pred_proba = clf.predict_proba(valid_embeddings.cpu().detach().numpy())[:,1]
test_pred_proba = clf.predict_proba(test_embeddings.cpu().detach().numpy())[:,1]

In [ ]:
from sklearn.metrics import ndcg_score



ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_pred_proba
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

<ipython-input-37-4d8ac00adc28>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_pred_proba
<ipython-input-37-4d8ac00adc28>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


In [ ]:
np.mean(ndcg_score_all)

0.9964604106396241

# Train Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=60, random_state=0).fit(train_embeddings.cpu().detach().numpy(), train['label'].tolist())
valid_pred_proba = clf.predict_proba(valid_embeddings.cpu().detach().numpy())[:,1]
test_pred_proba = clf.predict_proba(test_embeddings.cpu().detach().numpy())[:,1]
from sklearn.metrics import ndcg_score



ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_pred_proba
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

print(np.mean(ndcg_score_all))

0.9956667640469065


<ipython-input-43-e29a6cf71e06>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_pred_proba
<ipython-input-43-e29a6cf71e06>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


# Train XGB Classifier

In [ ]:
!pip install -q xgboost

In [ ]:
import xgboost as xgb
clf = xgb.XGBClassifier(n_estimators=60, random_state=0).fit(train_embeddings.cpu().detach().numpy(), train['label'].tolist(), eval_set=[(valid_embeddings.cpu().detach().numpy(), valid['label'].tolist())])
valid_pred_proba = clf.predict_proba(valid_embeddings.cpu().detach().numpy())[:,1]
test_pred_proba = clf.predict_proba(test_embeddings.cpu().detach().numpy())[:,1]
from sklearn.metrics import ndcg_score



ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_pred_proba
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

print(np.mean(ndcg_score_all))

[0]	validation_0-logloss:0.86239
[1]	validation_0-logloss:0.85353
[2]	validation_0-logloss:0.89696
[3]	validation_0-logloss:0.90061
[4]	validation_0-logloss:0.93766
[5]	validation_0-logloss:0.93027
[6]	validation_0-logloss:0.98198
[7]	validation_0-logloss:1.00424
[8]	validation_0-logloss:1.04247
[9]	validation_0-logloss:1.05902
[10]	validation_0-logloss:1.07769
[11]	validation_0-logloss:1.09421
[12]	validation_0-logloss:1.10607
[13]	validation_0-logloss:1.08886
[14]	validation_0-logloss:1.06283
[15]	validation_0-logloss:1.05850
[16]	validation_0-logloss:1.07166
[17]	validation_0-logloss:1.09362
[18]	validation_0-logloss:1.10556
[19]	validation_0-logloss:1.11826
[20]	validation_0-logloss:1.16090
[21]	validation_0-logloss:1.19609
[22]	validation_0-logloss:1.20716
[23]	validation_0-logloss:1.20787
[24]	validation_0-logloss:1.22526
[25]	validation_0-logloss:1.24383
[26]	validation_0-logloss:1.24552
[27]	validation_0-logloss:1.24616
[28]	validation_0-logloss:1.25500
[29]	validation_0-loglos

<ipython-input-45-aa1be1be29fc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_pred_proba
<ipython-input-45-aa1be1be29fc>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


# Train Linear Regressor

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression().fit(train_embeddings.cpu().detach().numpy(), train['mean_target_percentage_increment'].tolist())
valid_pred_proba = clf.predict(valid_embeddings.cpu().detach().numpy())
test_pred_proba = clf.predict(test_embeddings.cpu().detach().numpy())
from sklearn.metrics import ndcg_score



ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_pred_proba
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

print(np.mean(ndcg_score_all))

0.9955079843244024


<ipython-input-48-f6a5e0e77664>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_pred_proba
<ipython-input-48-f6a5e0e77664>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


# Train Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=60, random_state=0).fit(train_embeddings.cpu().detach().numpy(), train['mean_target_percentage_increment'].tolist())
valid_pred_proba = clf.predict(valid_embeddings.cpu().detach().numpy())
test_pred_proba = clf.predict(test_embeddings.cpu().detach().numpy())
from sklearn.metrics import ndcg_score



ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_pred_proba
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

print(np.mean(ndcg_score_all))

0.9960644543859347


<ipython-input-49-e2ef5456ff62>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_pred_proba
<ipython-input-49-e2ef5456ff62>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


# Train XGB Regressor

In [ ]:
import xgboost as xgb
clf = xgb.XGBRegressor(n_estimators=60, random_state=0).fit(train_embeddings.cpu().detach().numpy(), train['mean_target_percentage_increment'].tolist(), eval_set=[(valid_embeddings.cpu().detach().numpy(), valid['mean_target_percentage_increment'].tolist())])
valid_pred_proba = clf.predict(valid_embeddings.cpu().detach().numpy())
test_pred_proba = clf.predict(test_embeddings.cpu().detach().numpy())
from sklearn.metrics import ndcg_score



ndcg_score_all = []

dates = test['date'].unique()
test['predicted_probability'] = test_pred_proba
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

print(np.mean(ndcg_score_all))

[0]	validation_0-rmse:0.02469
[1]	validation_0-rmse:0.02716
[2]	validation_0-rmse:0.02723
[3]	validation_0-rmse:0.02936
[4]	validation_0-rmse:0.03026
[5]	validation_0-rmse:0.03106
[6]	validation_0-rmse:0.03105
[7]	validation_0-rmse:0.03138
[8]	validation_0-rmse:0.03182
[9]	validation_0-rmse:0.03225
[10]	validation_0-rmse:0.03196
[11]	validation_0-rmse:0.03185
[12]	validation_0-rmse:0.03242
[13]	validation_0-rmse:0.03213
[14]	validation_0-rmse:0.03199
[15]	validation_0-rmse:0.03218
[16]	validation_0-rmse:0.03225
[17]	validation_0-rmse:0.03252
[18]	validation_0-rmse:0.03239
[19]	validation_0-rmse:0.03264
[20]	validation_0-rmse:0.03261
[21]	validation_0-rmse:0.03266
[22]	validation_0-rmse:0.03258
[23]	validation_0-rmse:0.03256
[24]	validation_0-rmse:0.03257
[25]	validation_0-rmse:0.03252
[26]	validation_0-rmse:0.03254
[27]	validation_0-rmse:0.03262
[28]	validation_0-rmse:0.03265
[29]	validation_0-rmse:0.03264
[30]	validation_0-rmse:0.03265
[31]	validation_0-rmse:0.03263
[32]	validation_0-

<ipython-input-50-87c8070d20b6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_pred_proba
<ipython-input-50-87c8070d20b6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax


# Train Learning to Rank XGB Ranker

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder to transform the target variable to integers
label_encoder = LabelEncoder()

query_id_train = label_encoder.fit_transform(train['date'])
query_id_valid = label_encoder.fit_transform(valid['date'])
query_id_test = label_encoder.fit_transform(test['date'])

In [ ]:



clf =  xgb.XGBRanker(objective="rank:ndcg", learning_rate = 0.1, n_estimators = 100, random_state = 0)
clf.fit(train_embeddings.cpu().detach().numpy(), train['rank'].tolist(),qid=query_id_train)




valid_pred = clf.predict(valid_embeddings.cpu().detach().numpy())
test_pred = clf.predict(test_embeddings.cpu().detach().numpy())


In [ ]:
dates = test['date'].unique()
test['predicted_probability'] = test_pred
test['mean_target_percentage_increment_transformed'] = test_scores_softmax

for date in dates:
  test_sub = test[test['date'] == date]
  true_relevance = np.asarray([test_sub['mean_target_percentage_increment_transformed'].to_list()])
  # we predict some scores (relevance) for the answers
  predicted_scores = np.asarray([test_sub['predicted_probability'].to_list()])
  ndcg_score_all.append(ndcg_score(true_relevance, predicted_scores))

print(np.mean(ndcg_score_all))

0.9940487107469504


<ipython-input-63-d0f10bd2adb2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['predicted_probability'] = test_pred
<ipython-input-63-d0f10bd2adb2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['mean_target_percentage_increment_transformed'] = test_scores_softmax
